# Collecting Raw Data

In [2]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy
import matplotlib
import matplotlib.pyplot as plt
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import random
import sys
from IPython.display import display
from bs4 import BeautifulSoup
from IPython.display import display

In [101]:
# import user ids
users = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/user_ids_sep29')

# put user ids in list
user_id_list = users['0'].unique()
len(user_id_list)

15692

In [18]:
# init DataFrames 

revs_df = pd.DataFrame()
users_df = pd.DataFrame()
hostels_df = pd.DataFrame()

# Scrape for Users and Revs Data

In [32]:
# Get details for user and rev DFs

for i,user in enumerate(user_id_list):
    
    rv_df = pd.DataFrame()
    us_df = pd.DataFrame()
    X = 'http://www.hostelworld.com/profile/' + str(user) + '/reviews'
    response = requests.get(X)
    HTML = response.text
    
    rv_df['text'] = Selector(text=HTML).xpath('//div[@id]/p/text()').extract()
    rv_df['user_id'] = [user]*len(rv_df['text'])
    rv_df['rev_date'] = Selector(text=HTML).xpath("//span[@class='reviewdate']/text()").extract()
    rv_df['score'] = Selector(text=HTML).xpath("//div[@class='textrating']/text()").extract()
    rv_df['hostel'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/text()").extract()
    rv_df['location'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/span/text()").extract()
    rv_df['link'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/@href").extract()

    us_df['user_id'] = [user]
    us_df['about'] = [Selector(text=HTML).xpath("//li[@class='reviewerdetails']/text()").extract()[0]] 
    us_df['num_revs']=len(rv_df['text']) 
  
    # prints out progress
    if i%10 == 0:
        sys.stdout.write('\r'+ str((float(i)/len(user_id_list))*100)+'%')
     
    users_df = users_df.append(us_df,ignore_index=True)
    revs_df = revs_df.append(rv_df,ignore_index=True)
    

99.8260113093%

In [33]:

users_df.shape, revs_df.shape

((15777, 3), (33494, 7))

In [28]:

display("users_df.head(2)",users_df.head(2),'revs_df.head(2)',revs_df.head(2))

'users_df.head(2)'

,user_id,about,num_revs
0,4929738,\n ...,1
1,3151081,\n ...,2


'revs_df.head(2)'

,text,user_id,rev_date,score,hostel,location,link
0,I really enjoyed staying at this hostel.That w...,4929738,29th Feb 2008,10.0,Smile Hostal,"Lisbon, Portugal",http://www.hostelworld.com/hosteldetails.php/S...
1,excellent hostel. definitely planning on retu...,3151081,5th Aug 2006,10.0,Alberguinn Youth Hostel,"Barcelona, Spain",http://www.hostelworld.com/hosteldetails.php/A...


In [94]:
# save user and revs dfs

users_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_users_sep29_4.csv', encoding='utf8')
revs_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_revs_sep29_4.csv',encoding='utf8')

# Scrape for Hostels Data

In [79]:
# Get data for hostels 

%time
# List to collect hostels that no longer exist
hostels_DNE = []

for i,url in enumerate(revs_df.link):

    hst_df = pd.DataFrame()
    X = str(url)
    response = requests.get(X)
    HTML = response.text
    
    free = Selector(text=HTML).xpath('//li[1]/div/p/text()').extract()
    general = Selector(text=HTML).xpath('//li[2]/div/p').extract()
    services = Selector(text=HTML).xpath('//li[3]/div/p').extract()
    food_drink = Selector(text=HTML).xpath('//li[4]/div/p').extract()
    entertainment = Selector(text=HTML).xpath('//li[5]/div/p').extract()
    policies = Selector(text=HTML).xpath("//div[@id='panel4']/p").extract()
    ratings = Selector(text=HTML).xpath("//div[@class='ratingbox']/text()").extract()
    description = Selector(text=HTML).xpath("//div[@class='section-propertytext']/div[@class='prop-intro']/div[@class='microdetailstext hiddentext prop-text bigtext ']/text()").extract()
    awards = len(Selector(text=HTML).xpath("//ul[@class='small-block-grid-1 medium-block-grid-2 large-block-grid-3']/li/b/text()").extract())
    num_rev = str(Selector(text=HTML).xpath("//div[@class='microsite_rating clearfix']/div/span[@class='reviews-count']/text()").extract()).split(" ")[0]
    rev_hl = Selector(text=HTML).xpath("//div[@class='small-12 medium-4 large-3 columns hide-for-small']/div/ul/li/span[@class='ratingfactorlabel']").extract()
    
    hst = revs_df.hostel.iloc[i]
    loc = revs_df.location.iloc[i]
    
    # prints out progress
    if i%10 == 0:
        sys.stdout.write('\r'+ str((float(i)/len(revs_df.link))*100)+'%')
        
    # if statements to make sure the link goes to a hostel that still exist
    if len(ratings) != 0:
        if float(ratings[0]) != 0.0:

            hst_df['rating'] = [float(ratings[0])]
            hst_df['desc'] = [description]
            hst_df['FREE'] = [free]
            hst_df['GENERAL'] = [general] 
            hst_df['SERVICES'] = [services]
            hst_df['FOOD_DRINK'] = [food_drink]
            hst_df['ENTERTAINMENT'] = [entertainment]
            hst_df['POLICIES'] = [policies]
            hst_df['hostel']= hst
            hst_df['location']= loc
            hst_df['num_revs'] = num_rev
            hst_df['num_awards'] = awards
            hst_df['rev_highlights'] = [rev_hl]
        
            hostels_df = hostels_df.append(hst_df,ignore_index=True)
            
        # else statment to collect index where hostles no longer exist
        else:
            hostels_DNE.append(i)
    # else statment to collect index where hostles no longer exist
    else:
        hostels_DNE.append(i)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs
99.922796956%

In [83]:
hostels_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_hostels_oct4.csv',encoding='utf8')

In [82]:

users_df.shape, revs_df.shape, hostels_df.shape

((15613, 4), (33183, 8), (23003, 13))

## Drop duplicates and Hostels that DNE
- Del reviews coresponding to hostels that do not exist 
- Del users that only reviewed hostels that do not exist
- Drop duplicates from hostels

In [84]:
# get rid of rows in revs df that corespond to hostels in hostels_DNE list

hostels_E =[i for i in revs_df.index if i not in hostels_DNE] 
revs_df_exist = revs_df.loc[revs_df.index.isin(hostels_E)].reset_index()
del revs_df_exist['index']

In [88]:
# del users that only reviewed hostels that no longer exist

revs_exist_uid =[u for u in revs_df_exist.user_id]
users_df_revE = users_df.loc[users_df.user_id.isin(revs_exist_uid)].reset_index()
del users_df_revE['index']

In [89]:
# new revs and hostels df should be same length

users_df_revE.shape, revs_df_exist.shape, hostels_df.shape

((11498, 4), (23003, 8), (23003, 13))

In [105]:
# look at hostel repeats

len(revs_df_exist.hostel.unique())

5642

In [114]:
# create a new hostels df without repeats

hostles_df2 = hostels_df.drop_duplicates(['hostel'])
hostles_df2.head(2)

,rating,desc,FREE,GENERAL,SERVICES,FOOD_DRINK,ENTERTAINMENT,POLICIES,hostel,location,num_revs,num_awards,rev_highlights
0,2.0,[\n Stay with us near to th...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[],Fortuna Botel,"Budapest, Hungary",[],0,[]
1,7.5,[\n The Equity Point Sea is...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,"[<p>\n Age Restriction<i class=""fa ...",Equity Point Sea,"Barcelona, Spain",[],0,"[<span class=""ratingfactorlabel"">Perfect <b>Lo..."


In [116]:
# save final raw user, rev, and hostel DFs

users_df_revE.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_users_oct4E.csv',encoding='utf8')
hostles_df2.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_hostels_oct4E_ND.csv',encoding='utf8')
revs_df_exist.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_revs_oct4E.csv',encoding='utf8')